<a href="https://colab.research.google.com/github/AchrefHamdi/House-Price-Predection/blob/main/House_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn==1.0.2

In [ ]:
!pip install pycaret

In [4]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv")
dataset.head()

In [26]:
i=dataset

In [27]:
i.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [ ]:
dataset.ocean_proximity.value_counts()

In [ ]:
import matplotlib.pyplot as plt
dataset.hist(bins=50, figsize=(10, 8))
plt.show()

In [9]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
dataset['income_cat'] = pd.cut(dataset['median_income'], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
dataset['income_cat'].hist()
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(dataset, dataset["income_cat"]):
    strat_train_set = dataset.loc[train_index]
    strat_test_set = dataset.loc[test_index]
print(strat_test_set['income_cat'].value_counts() / len(strat_test_set))

In [12]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)
dataset = strat_train_set.copy()

In [ ]:
dataset.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4, s=dataset['population']/100, label='population',
figsize=(12, 8), c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)
plt.legend()
plt.show()

In [ ]:
corr_matrix = dataset.corr()
print(corr_matrix.median_house_value.sort_values(ascending=False))

In [ ]:
dataset["rooms_per_household"] = dataset["total_rooms"]/dataset["households"]
dataset["bedrooms_per_room"] = dataset["total_bedrooms"]/dataset["total_rooms"]
dataset["population_per_household"] = dataset["population"]/dataset["households"]
corr_matrix = dataset.corr()
print(corr_matrix["median_house_value"].sort_values(ascending=False))

In [19]:
dataset = strat_train_set.drop("median_house_value", axis=1)
dataset_labels = strat_train_set["median_house_value"].copy()

median = dataset["total_bedrooms"].median()
dataset["total_bedrooms"].fillna(median, inplace=True)

dataset_num = dataset.drop("ocean_proximity", axis=1)

from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
num_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])
dataset_num_tr = num_pipeline.fit_transform(dataset_num)

from sklearn.compose import ColumnTransformer
num_attribs = list(dataset_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])
dataset_prepared = full_pipeline.fit_transform(dataset)

In [21]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(dataset_prepared, dataset_labels)

data = dataset.iloc[:3]
labels = dataset_labels.iloc[:3]
data_preparation = full_pipeline.transform(data)
print("Predictions: ", model.predict(data_preparation))

Predictions:  [210644.60459286 317768.80697211 210956.43331178]


In [22]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(dataset_prepared, dataset_labels)

data = dataset.iloc[:3]
labels = dataset_labels.iloc[:3]
data_preparation = full_pipeline.transform(data)
print("Predictions: ", model.predict(data_preparation))

Predictions:  [175800. 234500. 120700.]


In [ ]:
dataset.info()

In [35]:
from sklearn.svm import SVC
model = SVC()
model.fit(dataset_prepared, dataset_labels)

data = dataset.iloc[:3]
labels = dataset_labels.iloc[:3]
data_preparation = full_pipeline.transform(data)
print("Predictions: ", model.predict(data_preparation))

Predictions:  [137500. 500001. 112500.]


In [ ]:
import pycaret.classification 
from pycaret.classification import *
clf = setup(dataset, target = "median_house_value", silent = True, session_id = 786)
compare_models()
from google.colab import data_table
from vega_datasets import data
data_table.enable_dataframe_formatter()

clf.airports()

,Description,Value
0,session_id,786
1,Target,median_house_value
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(20640, 10)"
5,Missing Values,True
6,Numeric Features,8
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


IntProgress(value=0, description='Processing: ', max=74)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:58:47
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
